<a href="https://colab.research.google.com/github/FranciscoAguiar/DataScienceProgramming/blob/master/XGBoost_Feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score, classification_report
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [2]:
dataset = pd.read_csv('combined.csv', sep=',')

In [3]:
dataset.dtypes

id           int64
feedback    object
classe       int64
dtype: object

In [4]:
dataset.head(5)

,id,feedback,classe
0,23,Ler outros textos sobre o assunto para lher da...,0
1,24,Procure ler mais; para escrever mais acerca do...,0
2,35,Texto bom; precisa apenas ler mais um pouco; p...,0
3,37,Ter cuidado com copias da internet.,0
4,68,Carssimo; o trabalho precisa de uma sistematiz...,1


In [6]:
le = LabelEncoder()
dataset["feedback"] = le.fit_transform(dataset["feedback"].astype(str))

In [7]:
str_cols = dataset.columns[dataset.columns.str.contains('(?:feedback)')]
clfs = {c:LabelEncoder() for c in str_cols}

for col, clf in clfs.items():
    dataset[col] = clfs[col].fit_transform(dataset[col])

In [8]:
# split data into X and y
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]

In [9]:
X.shape

(1000, 2)

In [10]:
Y.shape

(1000,)

In [11]:
X

,id,feedback
0,23,389
1,24,635
2,35,732
3,37,726
4,68,236
...,...,...
995,13809,648
996,13810,650
997,13811,713
998,13813,720


In [15]:
def erro_por_classe(matriz_confusao, resultados):
    # A PARTIR DA MATRIZ DE CONFUSÃO CALCULA O ERRO POR CLASSE
    # **
    # matriz_confusao - soma das matrize de confusão de cada fold do tipo ndarray(numpy) de forma NxN
    # resultados - dicionário de todos os resultados
    # **
    # retorna o dicionário resultados com os erros de cada classe preenchidos

    tam = matriz_confusao.shape[0]

    for i in range(tam):

        acerto = matriz_confusao[i][i]
        total = sum(matriz_confusao[i])

        taxa_erro = round(1 - (acerto / total),4)

        resultados["erro_classe_"+str(i)].append(taxa_erro)

In [16]:
def validacao_cruzada(X, y, features, k, ntree, resultados ):
    ##É REALIZADO OS O EXPERIMENTO COM VALIDAÇÃO CRUZADA E OS RESULTADOS É ADICIONADO A UM DICIONÁRIO
    # **
    # X - dados
    # y - classes
    # k - número de folds
    # ntree - Número de árvores
    # mtry - número de features
    # metricas - lista de metricas que serão utilizadas na avaliacão( "acurácia","kappa", "OOB_erro")
    # resultados - dicionário que vai ser utilizado para cada experimento, salvando os resultados em um dicionário para ser salvo em CSV
    # **
    # retorna o dicionário resultados com os resultados desse experimento adicionados

    resultados_parciais = {} #SALVAR RESULTADOS DE CADA RODADA DA VALIDAÇÃO CRUZADA
    resultados_parciais.update({'ntree': []})
    resultados_parciais.update({'mtry': []})
    resultados_parciais.update({'acurácia': []})
    resultados_parciais.update({'kappa': []})
    resultados_parciais.update({'accuracy': []})
    resultados_parciais.update({'erro': []})

    ## VALIDAÇÃO CRUZADA

    rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=54321) #DIVIDI OS DADOS NOS CONJUNTOS QUE SERÃO DE      TREINO E TESTE EM CADA RODADA DA VALIDAÇÃO CRUZZADA

    matriz_confusao = np.zeros((2,2))


    for train_index, test_index in rkf.split(X, y):
        X_train, X_test = [X.iloc[i] for i in train_index], [X.iloc[j] for j in test_index]
        y_train, y_test = [y.iloc[i] for i in train_index], [y.iloc[j] for j in test_index]

        X_train_np = np.asarray(X_train)
        X_test_np = np.asarray(X_test)
        y_train_np = np.asarray(y_train)
        y_test_np = np.asarray(y_test)

        classificador = XGBClassifier(n_estimators=ntree)
        classificador.fit(X_train_np, y_train_np)
        y_pred = classificador.predict(X_test_np)

        resultados_parciais["acurácia"].append(accuracy_score(y_pred, y_test_np))
        resultados_parciais["kappa"].append(cohen_kappa_score(y_pred, y_test_np))

        matriz_confusao = matriz_confusao + confusion_matrix(y_pred=y_pred, y_true=y_test_np) ##A MATRIZ DE CONFUSÃO FINAL SERÁ A SOMA DAS MATRIZES DE CONFUSÃO DE CADA RODADA DO KFOLD


    ## SALVANDO OS PARÊMTROS E RESULTADOS DO EXPERIMENTO


    #print(matriz_confusao)
    resultados['ntree'].append(classificador.n_estimators)
    erro_por_classe(matriz_confusao, resultados)

    media = np.mean(resultados_parciais["acurácia"])
    std = np.std(resultados_parciais["acurácia"])
    resultados["acurácia"].append(str(round(media,4))+"("+str(round(std,4))+")")

    resultados["accuracy"].append(round(media, 4))
    resultados["erro"].append(round(1 - media, 4))

    media = np.mean(resultados_parciais["kappa"])
    std = np.std(resultados_parciais["kappa"])
    resultados["kappa"].append(str(round(media, 4)) + "(" + str(round(std, 4)) + ")")



    return resultados, classificador


In [17]:
features = dataset.columns.difference(['id_feedback','class'])
print(features)


Index(['classe', 'feedback', 'id'], dtype='object')


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)


In [20]:
resultados = {}
resultados.update({'ntree': []})
resultados.update({'mtry': []})
resultados.update({'acurácia': []})
resultados.update({'kappa': []})
resultados.update({'accuracy': []})
resultados.update({'erro': []})

In [21]:
classes = set(y_train)
for i in classes:
    resultados.update({"erro_classe_"+str(i):[]})


In [23]:
j = 0
maior = 0.0
best_mtree = 0

##NTREE - TREINAMENTO
for i in range(100,601,50):
   #mtry = randint(5, 116)

  resultados, classificador = validacao_cruzada(X_train, y_train, features, k=10, ntree=i, resultados=resultados)

  if resultados["accuracy"][j] > maior:
    maior = resultados["accuracy"][j]
    best_classifier = classificador
    best_mtree = i
  j = j+1

In [25]:
def printResults(vector):
    for element in vector:
        print(element)

In [26]:
print("RESULTADOS: \n")
print(resultados)

printResults(resultados['acurácia'])
printResults(resultados['kappa'])

RESULTADOS: 

{'ntree': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600], 'mtry': [], 'acurácia': ['0.89(0.005)', '0.8862(0.0073)', '0.8838(0.0161)', '0.88(0.015)', '0.8875(0.0168)', '0.8788(0.0184)', '0.8775(0.02)', '0.875(0.0224)', '0.8762(0.0218)', '0.8775(0.02)', '0.8788(0.0196)'], 'kappa': ['0.027(0.0486)', '0.0528(0.0441)', '0.0945(0.0622)', '0.0855(0.0566)', '0.1552(0.0991)', '0.1352(0.0985)', '0.1334(0.1008)', '0.1292(0.1039)', '0.141(0.1195)', '0.155(0.1049)', '0.1691(0.101)'], 'accuracy': [0.89, 0.8862, 0.8838, 0.88, 0.8875, 0.8788, 0.8775, 0.875, 0.8762, 0.8775, 0.8788], 'erro': [0.11, 0.1138, 0.1162, 0.12, 0.1125, 0.1212, 0.1225, 0.125, 0.1237, 0.1225, 0.1212], 'erro_classe_0': [0.9765, 0.9529, 0.9176, 0.9176, 0.8706, 0.8706, 0.8706, 0.8706, 0.8588, 0.8471, 0.8353], 'erro_classe_1': [0.007, 0.014, 0.021, 0.0252, 0.0224, 0.0322, 0.0336, 0.0364, 0.0364, 0.0364, 0.0364]}
0.89(0.005)
0.8862(0.0073)
0.8838(0.0161)
0.88(0.015)
0.8875(0.0168)
0.8788(0.0184)
0.8775(0.02)
0.8

In [ ]:
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=54321) #DIVIDI OS DADOS NOS CONJUNTOS QUE SERÃO DE      TREINO E TESTE EM CADA RODADA DA VALIDAÇÃO CRUZZADA


In [ ]:
        X_train, X_test = [X.iloc[i] for i in train_index], [X.iloc[j] for j in test_index]
        y_train, y_test = [Y.iloc[i] for i in train_index], [Y.iloc[j] for j in test_index]

        X_train_np = np.asarray(X_train)
        X_test_np = np.asarray(X_test)
        y_train_np = np.asarray(y_train)
        y_test_np = np.asarray(y_test)

       
       

In [ ]:
model = XGBClassifier()
model.fit(X_train_np, y_train_np)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 66.67%
